In [18]:
import matplotlib.pyplot as plt
import scipy
import numpy as np
import networkx as nx
import torch
import gzip

import os
import random

Importing data

In [19]:
data_dir = os.path.join(os.getcwd(), 'data')
file_names = [
    "higgs-activity_time.txt.gz",
    "higgs-mention_network.edgelist.gz",
    "higgs-reply_network.edgelist.gz",
    "higgs-retweet_network.edgelist.gz",
    "higgs-social_network.edgelist.gz",
]

In [20]:
mentions: nx.classes.digraph.DiGraph = nx.read_weighted_edgelist(os.path.join(data_dir, file_names[1]), create_using=nx.DiGraph)
print(f"Loaded {file_names[1]}")
replys: nx.classes.digraph.DiGraph = nx.read_weighted_edgelist(os.path.join(data_dir, file_names[2]), create_using=nx.DiGraph)
print(f"Loaded {file_names[2]}")
retweets: nx.classes.digraph.DiGraph = nx.read_weighted_edgelist(os.path.join(data_dir, file_names[3]), create_using=nx.DiGraph)
print(f"Loaded {file_names[3]}")
# social = nx.read_edgelist(os.path.join(data_dir, file_names[4]), create_using=nx.DiGraph) # This file is around ~200MB, so I recommend only loading it by itself. It takes me ~4 minutes to load, and slows down the rest of the script when in-memory.

Loaded higgs-mention_network.edgelist.gz
Loaded higgs-reply_network.edgelist.gz
Loaded higgs-retweet_network.edgelist.gz


In [10]:
activity_times = [] # Use this how you wish.
with gzip.open(os.path.join(data_dir, file_names[0]), 'rt') as f:
    for line in f:
        activity_times.append(line.strip())
activity_times[:3]

['223789 213163 1341100972 MT',
 '223789 213163 1341100972 RE',
 '376989 50329 1341101181 RT']

Henry's Analysis Section:

In [ ]:
# TODO

Ryan's Analysis Section:

In [4]:
# TODO